In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 

import seaborn as sns # used for plot interactive graph.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC  
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import sklearn.metrics.cluster as smc
from matplotlib import pyplot

import os
import itertools
import random 

%matplotlib inline

random.seed(148) 

In [2]:
# Helper function that allows you to draw nicely formatted confusion matrices
def draw_confusion_matrix(y, yhat, classes):
    '''
        Draws a confusion matrix for the given target and predictions
        Adapted from scikit-learn example.
    '''
    plt.cla()
    plt.clf()
    matrix = confusion_matrix(y, yhat)
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    num_classes = len(classes)
    plt.xticks(np.arange(num_classes), classes, rotation=90)
    plt.yticks(np.arange(num_classes), classes)
    
    fmt = 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [7]:
avgPrice   = pd.read_csv('./data/BrandAverageRetailPrice.csv')
details    = pd.read_csv('./data/BrandDetails.csv')
totalSales = pd.read_csv('./data/BrandTotalSales.csv')
totalUnits = pd.read_csv('./data/BrandTotalUnits.csv')

In [35]:
#analyze avg price
print(avgPrice.head())
print(avgPrice.describe())
print(avgPrice.info())

print(len(avgPrice['Brands'].unique()))

         Brands   Months        ARP  vs. Prior Period
0  #BlackSeries  08/2020  15.684913               NaN
1  #BlackSeries  09/2020        NaN         -1.000000
2  #BlackSeries  01/2021  13.611428               NaN
3  #BlackSeries  02/2021  11.873182         -0.127705
4  #BlackSeries  03/2021        NaN         -1.000000
                ARP  vs. Prior Period
count  25279.000000      24499.000000
mean      22.679732         -0.065028
std       19.802724          0.388923
min        0.000000         -1.000000
25%       10.512827         -0.088073
50%       17.033051         -0.011649
75%       31.505612          0.045232
max      700.874984         12.645741
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27211 entries, 0 to 27210
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brands            27211 non-null  object 
 1   Months            27211 non-null  object 
 2   ARP               25279 non-null 

In [56]:
# print(details.head())
print(details.describe())
print(details.info())
#details['Channel'].unique() #can probably drop this column
len(details['Brand'].unique()) #1123 different brands
# print("l1: ", 
# details['Category L1'].unique(), "l2: ",
# details['Category L2'].unique(), "l3: ",
# details['Category L3'].unique(), "l4: ",
# details['Category L4'].unique(), "l5: ",
# details['Category L5'].unique(),)

details

                 ARP  Items Per Pack
count  144977.000000   144977.000000
mean       30.828439        1.938259
std        19.367580       17.294108
min         0.000000        0.000000
25%        16.407796        0.000000
50%        28.073823        0.000000
75%        41.781699        0.000000
max       874.800010     1000.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144977 entries, 0 to 144976
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State                144977 non-null  object 
 1   Channel              144977 non-null  object 
 2   Category L1          144977 non-null  object 
 3   Category L2          144977 non-null  object 
 4   Category L3          144245 non-null  object 
 5   Category L4          102618 non-null  object 
 6   Category L5          50135 non-null   object 
 7   Brand                144977 non-null  object 
 8   Product Description  144977 non-null  obj

,State,Channel,Category L1,Category L2,Category L3,Category L4,Category L5,Brand,Product Description,Total Sales ($),...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
0,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),"1,103.964857",...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
1,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),674.645211,...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
2,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"2,473.699102",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
3,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"14,589.916417",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
4,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,145.39627,...,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144972,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Live Resin,NaN,Zkittlez,Zkittlez - Lemon Drop - Live Resin,"3,051.854478",...,0,0,THC Only,NaN,Lemon Drop,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$40.00 to $44.99
144973,California,Licensed,Inhaleables,Pre-Rolled,Pre-Rolled,NaN,NaN,Zkittlez,"Zkittlez - Blueberry Zkittlez - Pre-Rolled, 0....","6,776.219336",...,0,0,THC Only,NaN,Blueberry Zkittlez,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99
144974,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Live Resin,NaN,Zkittlez,Zkittlez - Lemon Drop - Live Resin,"4,341.808787",...,0,0,THC Only,NaN,Lemon Drop,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$45.00 to $49.99
144975,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Live Resin,NaN,Zkittlez,Zkittlez - Lemon Drop - Live Resin,"6,764.1640020000000",...,0,0,THC Only,NaN,Lemon Drop,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$60.00 to $64.99


In [29]:
totalSales.head()

,Months,Brand,Total Sales ($)
0,09/2018,10x Infused,"1,711.334232"
1,09/2018,1964 Supply Co.,"25,475.21594500000"
2,09/2018,3 Bros Grow,"120,153.644757"
3,09/2018,3 Leaf,"6,063.5297850000000"
4,09/2018,350 Fire,"631,510.0481550000"


In [30]:
totalSales.describe()

,Months,Brand,Total Sales ($)
count,25279,25279,25279
unique,37,1627,25277
top,05/2021,Grizzly Peak Farms (CA),0
freq,848,37,3


In [31]:
totalSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Months           25279 non-null  object
 1   Brand            25279 non-null  object
 2   Total Sales ($)  25279 non-null  object
dtypes: object(3)
memory usage: 592.6+ KB


In [36]:
len(totalSales['Brand'].unique())

1627

In [32]:
totalUnits.head()

,Brands,Months,Total Units,vs. Prior Period
0,#BlackSeries,08/2020,"1,616.3390040000000",NaN
1,#BlackSeries,09/2020,NaN,-1.000000
2,#BlackSeries,01/2021,715.5328380000000,NaN
3,#BlackSeries,02/2021,766.669135,0.071466
4,#BlackSeries,03/2021,NaN,-1.000000


In [33]:
totalUnits.describe()

,vs. Prior Period
count,24935.000000
mean,0.265306
std,3.291373
min,-1.000000
25%,-0.351822
50%,-0.055216
75%,0.240113
max,250.792020


In [34]:
totalUnits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brands            27686 non-null  object 
 1   Months            27686 non-null  object 
 2   Total Units       25712 non-null  object 
 3   vs. Prior Period  24935 non-null  float64
dtypes: float64(1), object(3)
memory usage: 865.3+ KB


In [38]:
len(totalUnits['Brands'].unique())

1640

In [55]:
frames = [avgPrice, details, totalSales, totalUnits]

data1 = pd.merge(avgPrice, )


data = pd.merge(frames)

print(data.columns)



Index(['Brands', 'Months', 'ARP', 'vs. Prior Period', 'State', 'Channel',
       'Category L1', 'Category L2', 'Category L3', 'Category L4',
       'Category L5', 'Brand', 'Product Description', 'Total Sales ($)',
       'Total Units', 'Flavor', 'Items Per Pack', 'Item Weight', 'Total THC',
       'Total CBD', 'Contains CBD', 'Pax Filter', 'Strain', 'Is Flavored',
       'Mood Effect', 'Generic Vendor', 'Generic Items', '$5 Price Increment'],
      dtype='object')


,Brands,Months,ARP,vs. Prior Period,State,Channel,Category L1,Category L2,Category L3,Category L4,...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
0,#BlackSeries,08/2020,15.684913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#BlackSeries,09/2020,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#BlackSeries,01/2021,13.611428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#BlackSeries,02/2021,11.873182,-0.127705,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#BlackSeries,03/2021,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27681,Zuma Topicals,08/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27682,Zuma Topicals,09/2019,NaN,0.485707,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27683,Zuma Topicals,10/2019,NaN,-0.250370,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27684,Zuma Topicals,11/2019,NaN,-0.609484,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
